In [1]:
import sys, os
sys.path.append(f'../../code')
path_to_json = os.getcwd()

from IPython.display import display, Markdown

In [2]:
import json
def load_case(path):
    display(Markdown(f'## {path}'))
    with open(path, 'r+') as file: bundle = json.load(file)
    print('Temperature:', bundle['temperature'])
    return bundle

display(Markdown('# Files'))
json_files_mc = sorted([pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and pos_json.startswith('mc-hT')],
                       reverse = True)
json_files_alg1 = sorted([pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and pos_json.startswith('a1-hT')],
                         reverse = True)
json_files_alg2 = sorted([pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and pos_json.startswith('a2-hT')],
                         reverse = True)

json_files_theory = sorted([pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json') and pos_json.startswith('theory-hT')],
                         reverse = True)


print("MC: ", json_files_mc)
print("A1: ", json_files_alg1)
print("A2: ", json_files_alg2)
print("Theory: ", json_files_theory)


mc = load_case(json_files_mc[0])
alg1 = load_case(json_files_alg1[0])
alg2 = load_case(json_files_alg2[0])
theory = load_case(json_files_theory[0])

# Files

MC:  ['mc-hT.json']
A1:  ['a1-hT.json']
A2:  ['a2-hT.json']
Theory:  ['theory-hT.json']


## mc-hT.json

Temperature: 1


## a1-hT.json

Temperature: 1


## a2-hT.json

Temperature: 1


## theory-hT.json

Temperature: 1


In [3]:
def theory_dictionary(theory_quantity_levels, quantity_level_value):
    quantity_level = defaultdict(int)
    keys = theory_quantity_levels
    for k in zip(keys):
        quantity_level[k] = 0
    i = 0
    for k in quantity_level.keys():
        quantity_level[k] = quantity_level_value[i]
        i += 1
    quantity_level = {key[0]: value for key, value in quantity_level.items()}
    
    return quantity_level

In [4]:
import ising
from ising import *
import matplotlib.pyplot as plt

theory_engy_counts = np.array(theory["energy_probs"]) * len(mc["energy_sample"]);
theory_m_counts = np.array(theory["magnetization_probs"]) * len(mc["magnetization_sample"]);
theory_engy_dict = theory_dictionary(theory["energy_level"], theory_engy_counts)
theory_m_dict = theory_dictionary(theory["magnetization_level"], theory_m_counts)

mc_m_counts = count_variables(mc["magnetization_sample"])
mc_m_counts, theory_m_dict = equilibrate_counts(mc_m_counts,theory_m_dict)

alg1_m_counts = count_variables(alg1["magnetization_sample"])
alg1_m_counts, theory_m_dict = equilibrate_counts(alg1_m_counts,theory_m_dict)

alg2_m_counts = count_variables(alg2["magnetization_sample"])
alg2_m_counts, theory_m_dict = equilibrate_counts(alg2_m_counts,theory_m_dict)

mc_engy_counts = count_variables(mc["energy_sample"])
mc_engy_counts, theory_engy_dict = equilibrate_counts(mc_engy_counts,theory_engy_dict)

alg1_engy_counts = count_variables(alg1["energy_sample"])
alg1_engy_counts, theory_engy_dict = equilibrate_counts(alg1_engy_counts,theory_engy_dict)

alg2_engy_counts = count_variables(alg2["energy_sample"])
alg2_engy_counts, theory_engy_dict = equilibrate_counts(alg2_engy_counts,theory_engy_dict)

In [5]:
from statistical_tests import*
display(Markdown("# Perfromance chi2 test one sample"))
np.random.seed(0)
engy_multinomial = np.random.multinomial(np.sum(theory_engy_counts), np.array(theory["energy_probs"]))
m_multinomial = np.random.multinomial(np.sum(theory_m_counts), np.array(theory["magnetization_probs"]))
print(f' Performance of Chi2 on Energy:', one_sample_chi_squared_resampling(np.array(theory["energy_probs"]), engy_multinomial))
print(f' Performance of Chi2 on Magnetization:',  one_sample_chi_squared_resampling(np.array(theory["magnetization_probs"]), m_multinomial))

display(Markdown("# Chi2 test one sample"))
print(f'H0: the sample comes from the same distribution of the theoretical one')
mc_engy = np.array(list(mc_engy_counts.values()))
alg1_engy = np.array(list(alg1_engy_counts.values()))
alg2_engy = np.array(list(alg2_engy_counts.values()))
mc_m = np.array(list(mc_m_counts.values()))
alg1_m = np.array(list(alg1_m_counts.values()))
alg2_m = np.array(list(alg2_m_counts.values()))
print(f'Energy MC:', one_sample_chi_squared(np.array(theory["energy_probs"]), mc_engy))
print(f'Energy Alg1:', one_sample_chi_squared(np.array(theory["energy_probs"]), alg1_engy))
print(f'Energy Alg2:', one_sample_chi_squared(np.array(theory["energy_probs"]), alg2_engy))

print(f'Magnetization MC:', one_sample_chi_squared(np.array(theory["magnetization_probs"]), mc_m)) 
print(f'Magnetization Alg1:', one_sample_chi_squared(np.array(theory["magnetization_probs"]), alg1_m))
print(f'Magnetization Alg2:', one_sample_chi_squared(np.array(theory["magnetization_probs"]), alg2_m))

# Perfromance chi2 test one sample

 Performance of Chi2 on Energy: 0.91521
 Performance of Chi2 on Magnetization: 0.3068


# Chi2 test one sample

H0: the sample comes from the same distribution of the theoretical one
Energy MC: 0.9280830389741048
Energy Alg1: 0.9228276427942785
Energy Alg2: 0.8105741324180007
Magnetization MC: 0.29149910084151587
Magnetization Alg1: 0.05549537968549178
Magnetization Alg2: 0.8708992725698491


In [6]:
from statistical_tests import *
display(Markdown("# Loglikelihood ratio test one sample"))
print(f'H0: the sample comes from the same distribution of the theoretical one')
mc_engy = np.array(list(mc_engy_counts.values()))
alg1_engy = np.array(list(alg1_engy_counts.values()))
alg2_engy = np.array(list(alg2_engy_counts.values()))
mc_m = np.array(list(mc_m_counts.values()))
alg1_m = np.array(list(alg1_m_counts.values()))
alg2_m = np.array(list(alg2_m_counts.values()))
print(f'Energy MC:', one_sample_test(np.array(theory["energy_probs"]), mc_engy))
print(f'Energy Alg1:', one_sample_test(np.array(theory["energy_probs"]), alg1_engy))
print(f'Energy Alg2:', one_sample_test(np.array(theory["energy_probs"]), alg2_engy))
print(f'Magnetization MC:', one_sample_test(np.array(theory["magnetization_probs"]), mc_m)) 
print(f'Magnetization Alg1:', one_sample_test(np.array(theory["magnetization_probs"]), alg1_m))
print(f'Magnetization Alg2:', one_sample_test(np.array(theory["magnetization_probs"]), alg2_m))

# Loglikelihood ratio test one sample

H0: the sample comes from the same distribution of the theoretical one
Energy MC: 0.9999999996724576
Energy Alg1: 0.999999999723721
Energy Alg2: 0.9999996766896121
Magnetization MC: 0.27577444499621434
Magnetization Alg1: 0.051296280118694304
Magnetization Alg2: 0.8709503744223499


In [7]:
from statistical_tests import *
display(Markdown("# Chi2 test two sample"))
print(f'H0: the two sample comes from the same distribution')
mc_engy = np.array(list(mc_engy_counts.values()))
alg1_engy = np.array(list(alg1_engy_counts.values()))
alg2_engy = np.array(list(alg2_engy_counts.values()))
mc_m = np.array(list(mc_m_counts.values()))
alg1_m = np.array(list(alg1_m_counts.values()))
alg2_m = np.array(list(alg2_m_counts.values()))
print("MC - Alg1")
print(f'Energy :', two_sample_chi_squared(mc_engy, alg1_engy))
print(f'Magnetization :', two_sample_chi_squared(mc_m, alg1_m))
print("MC-Alg2")
print(f'Energy :', two_sample_chi_squared(mc_engy, alg2_engy))
print(f'Magnetization :', two_sample_chi_squared(mc_m, alg2_m))

# Chi2 test two sample

H0: the two sample comes from the same distribution
MC - Alg1
Energy : 0.9646949954337201
Magnetization : 0.22976842373218725
MC-Alg2
Energy : 0.7648487838510478
Magnetization : 0.26979483845891045
